In [1]:
from utilities import *
import torch
t = Tokenizer()

In [2]:
a = torch.tensor([[1, 0, 0],[0,0,0], [-2, -2, -2]])
b = torch.tensor([[-2,1,0],[0,1,1], [-1, 1, -1]])

In [3]:
x = t.tokenize(a)
y = t.tokenize(b)
print(a, b)
print(x, y)
w = t.detokenize(x)
x = t.detokenize(y)

print(w, x)

tensor([[ 3., 10., 50.],
        [ 2., 10., 50.],
        [ 0.,  0.,  0.]])
tensor([[ 0., 15., 50.],
        [ 2., 15., 75.],
        [ 1., 15., 25.]])
tensor([[ 1,  0,  0],
        [ 0,  0,  0],
        [-2, -2, -2]]) tensor([[-2,  1,  0],
        [ 0,  1,  1],
        [-1,  1, -1]])
tensor([63., 62.,  0.]) tensor([65., 92., 41.])
tensor([[ 1.,  8.,  0.],
        [ 0.,  8.,  0.],
        [-2., -2., -2.]]) tensor([[-2., 13.,  0.],
        [ 0., 13.,  1.],
        [-1., 13., -1.]])
